# <center>HW 4: Text preprocessing</center>

In [1]:
#NAme: Sahil Mahendra Mody
#Assignment:4
#CWID:20007262
#SUbject: Web mining

In [2]:
import nltk, re, json, string
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import numpy as np  
import pandas as pd
import string
from nltk.corpus import stopwords

import spacy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from spacy.lang.en import English

## Q1: Regular Expression

Suppose you have scraped the text shown below from an online source. You'd like to extract data using regular expression.

Define a **extract** function which:
- Takes a piece of text (in the format of shown below) as an input
- Extracts data into a list of tuples using regular expression, e.g.  `[('BTC-USD','56,212.15','-58.16','-0.10%'), ('ETH-USD',  ...), ...]`
- Returns the list of tuples

In [3]:
text='''Symbol   Last Price  Change   % Change   Note
                  BTC-USD  56,212.15   -58.16   -0.10%   Bitcoin 
                  ETH-USD  1,787.79    -53.63   -2.91%   Ether
                  BNB-USD  1,101,290.51      +5.81    +2.04%   Binance
                  USDT-USD 1.0003      -0.0004  -0.04%   Tether
                  ADA-USD  1.1187      -0.0528  -4.51%   Cardano
      '''
text

'Symbol   Last Price  Change   % Change   Note\n                  BTC-USD  56,212.15   -58.16   -0.10%   Bitcoin \n                  ETH-USD  1,787.79    -53.63   -2.91%   Ether\n                  BNB-USD  1,101,290.51      +5.81    +2.04%   Binance\n                  USDT-USD 1.0003      -0.0004  -0.04%   Tether\n                  ADA-USD  1.1187      -0.0528  -4.51%   Cardano\n      '

In [4]:
# Define the function
import re
def extract(text):
    #b=re.findall(r'(\w+-\w+)\s+(\d+.+\d%)',text)
    b=re.findall(r'(\w+-\w+)\s+([\d+,.]*)\s+([-\d+.]*)\s+([-\d+.%]*)',text)
    return b
# add your code

In [5]:
# Test the function
#lst_tuple = list(zip(lst1,lst2))
extract(text)

[('BTC-USD', '56,212.15', '-58.16', '-0.10%'),
 ('ETH-USD', '1,787.79', '-53.63', '-2.91%'),
 ('BNB-USD', '1,101,290.51', '+5.81', '+2.04%'),
 ('USDT-USD', '1.0003', '-0.0004', '-0.04%'),
 ('ADA-USD', '1.1187', '-0.0528', '-4.51%')]

## Q2: Collocation

Define a function `top_collocation(doc, K)` to find top-K collocations in specific patterns in a document as follows:
  - Takes a document (i.e. `doc`) and `K` as inputs
  - Find collocations as follows:
    - Tokenize the document and find POS tag of each token (hint: you can use NLTK word tokenizer or Spacy tokenizer).
    - Create bigrams from the tokens with POS tags and remove a bigram if one of the bigram is punctuation.

    - Keep only bigrams matching the following patterns:
       - `Adj + Noun`: e.g. linear function
       - `Noun + Noun`: e.g. regression coefficient
    - Get frequency of each bigram (hint: you can use nltk.FreqDist)
    - Returns top K collocations by frequency

In [6]:
# Define the function


def top_collocation(doc, K):
    #tokenizing
    tokens=nltk.word_tokenize(doc)
    punctuations=set(list(string.punctuation))
    #Striping punctuations
    strip=[token.strip() for token in tokens if token.lower() not in punctuations]

# tag each tokenized word
    tagged_tokens= nltk.pos_tag(strip)
    bigrams=list(nltk.bigrams(tagged_tokens))
    grams=[(x,y)for(x,y) in bigrams if x[0] not in punctuations and y[0] not in punctuations]
    #phrases1 for adjective + noun
    phrases1=[ (x[0],y[0]) for (x,y) in grams \
         if x[1].startswith('JJ') \
         and y[1].startswith('NN')]
    #phrases2 for noun + noun
    phrases2=[ (x[0],y[0]) for (x,y) in grams \
         if x[1].startswith('NN') \
         and y[1].startswith('NN')]
    phrases=phrases1+phrases2
    token_count=nltk.FreqDist(phrases)


    # add your code
    return token_count.most_common(K)
#     return result

In [7]:
data = json.load(open("C:/Users/SAHIL MODY/OneDrive/Desktop/Web Mining/qa.json","r"))
article = data["context"]

top_collocation(article, 6)

[(('public', 'health'), 14),
 (('community', 'spread'), 9),
 (('United', 'States'), 8),
 (('higher', 'risk'), 4),
 (('COVID-19', 'illness'), 4),
 (('elevated', 'risk'), 4)]

## Q3: Question and Answering (QA) System

Develop a QA system which allow you to search for answers in an article. For example, the file `qa.json` contains a research article. This article can answer a number of questions about COVID-19. You will design a solution to automatically search answers to these questions in this article.

`qa.json` is taken from https://github.com/deepset-ai/COVID-QA. This file contains a few questions, and answers to these questions have been located in the article. Let's define a QA system and check if your system can locate the right answers.

The following script helps you understand `qa.json`:

In [8]:
# Retrieve the article

data = json.load(open("C:/Users/SAHIL MODY/OneDrive/Desktop/Web Mining/qa.json","r"))
article = data["context"]

# A long article. Just print the first 200 characters
print(article[0:200])

CDC Summary 21 MAR 2020,
https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/summary.html

This is a rapidly evolving situation and CDC will provide updated information and guidance as it becomes 


In [9]:
# Retrieve all the questions and answers
qas = data["qas"]

# show the first question-answer pair. Note the answer starts at the 6117th character
print(qas[0])

# get all questions
qs = [item["question"] for item in qas]
qs

{'question': 'What age group has the highest rate of severe outcomes?', 'id': 236, 'answers': [{'text': 'people 85 years and older', 'answer_start': 6117}], 'is_impossible': False}


['What age group has the highest rate of severe outcomes?',
 'How is COVID-19 spread?',
 'How many states in the U.S. have reported cases of COVID-19?',
 'When did the White House launch the "15 Days to Slow the Spread" program?',
 'What should mildly-ill patients do?',
 'What type of virus is SARS-CoV-2?',
 'What viruses are similar to the COVID-19 coronavirus?',
 'What are the phases of a pandemic?',
 'At which phase does the peak of the pandemic occur?',
 'People with which medical conditions have a higher rate of severe illness?',
 'What kind of test can diagnose COVID-19?',
 'In what species did the COVID-19 virus likely originate?',
 'What risk factors should be considered in addition to clinical symptoms?']

Next, following the instructions below step by step to develop the QA system

### Q3.1. Tokenizer

Define a function `tokenize(doc)`  as follows:
   - Take a piece of text (i.e. variable `doc`) as an input
   - Split the input text into unigrams
   - Clean up tokens as follows:
       - Lemmatize all unigrams
       - Remove all stop words
       - Remove all punctuations
       - Convert all unigrams to the lower case 
       - remove empty unigrams
   - Return the list of unigrams after all the processing. (Hint: you can use spacy or stanza package for this task. To test if a token is stop word or punctuation, check https://spacy.io/api/token#attributes)

In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to C:\Users\SAHIL
[nltk_data]     MODY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Define the function

import string
def tokenize(doc):
    #tokens=doc.split(' ')
    punctuations="?:!.,;—"
    #tokeinizing
    sentence_words = nltk.word_tokenize(doc)
    hello=[]
    #sentence_words =[token.strip(string.punctuation+"—") for token in sentence_words]
    #removing punctuations
    for word in sentence_words:
        if word in punctuations:
            sentence_words.remove(word)
    tokens_without_sw = [word for word in sentence_words if not word in stopwords.words()]
    #converting to lower case
    for i in range(len(tokens_without_sw)):
        tokens_without_sw[i]=tokens_without_sw[i].lower()
    #Lemmatization
    from nltk.stem import WordNetLemmatizer
 
    lemmatizer = WordNetLemmatizer()
    for i in range(len(tokens_without_sw)) :
        a=lemmatizer.lemmatize(tokens_without_sw[i],pos='a')
        hello.append(a)
       
                       
    return hello

In [12]:
doc = 'Older people and people of all ages with severe chronic medical conditions — \
like heart disease, lung disease and diabetes, \
for example — seem to be at higher risk of developing serious COVID-19 illness.'

print(tokenize(doc))

['old', 'people', 'people', 'ages', 'severe', 'chronic', 'medical', 'conditions', 'like', 'heart', 'disease', 'lung', 'disease', 'diabetes', 'example', 'seem', 'high', 'risk', 'developing', 'serious', 'covid-19', 'illness']


### Q3.2. Compute TF-IDF Matrix

Define a function `compute_tfidf(docs)` as follows: 

- Take `docs`, a list of documents (e.g. a list of questions) as an input
- Tokenize each document in `docs` using the `tokenize` function defined in Q3.1. 
- Calculate tf_idf weights as shown in lecture notes (Hint: you can reuse the last code segment in NLP Lecture Notes (II))
- Return a smoothed normalized `tf_idf` array

In [13]:
# Define the function
def get_doc_tokens(doc):
    tokens=nltk.word_tokenize(doc.lower())
    stop_words = set(stopwords.words('english'))
    tokens_strip=[token.strip() \
            for token in tokens
            if token.strip() not in stop_words and token.strip() not in string.punctuation]
    
    # you can add bigrams, collocations, stemming, 
    # or lemmatization here
    
    token_count={token:tokens_strip.count(token) for token in set(tokens_strip)}
    return token_count

def compute_tfidf(docs):
    # step 2. process all documents to get list of token list
    docs_tokens={idx:get_doc_tokens(doc) \
             for idx,doc in enumerate(qs[0:3])}

    # step 3. get document-term matrix
    dtm=pd.DataFrame.from_dict(docs_tokens, orient="index" )
    dtm=dtm.fillna(0)
    dtm = dtm.sort_index(axis = 0)
      
    # step 4. get normalized term frequency (tf) matrix        
    tf=dtm.values
    doc_len=tf.sum(axis=1, keepdims=True)
    tf=np.divide(tf, doc_len)
    
    # step 5. get idf
    df=np.where(tf>0,1,0)
    #idf=np.log(np.divide(len(docs), \
    #    np.sum(df, axis=0)))+1

    smoothed_idf=np.log(np.divide(len(docs)+1, np.sum(df, axis=0)+1))+1    
    smoothed_tf_idf=normalize(tf*smoothed_idf)
    
    return smoothed_tf_idf

In [14]:
np.set_printoptions(precision=2)
compute_tfidf(qs[0:3])

array([[0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.8 , 0.61, 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.32, 0.42, 0.42, 0.42,
        0.42, 0.42]])

### Q3.2. Put Everything Together (bonus question)

Define a function `find_solutions(qs, article)` as follows: 

- Take two inputs:
    - `qs`: a list of questions (i.e. strings)
    - `article`: a document which may contain answers to the questions
- Segment the article into sentences (i.e. `sents`). You will locate the sentence which can answer a question.
- Concatenate the questions (`qs`) and sentences (`sents`) into a single list (i.e. `qs + sents`)
- Call the function `compute_tfidf` defined in Q3.2 with `qs + sents` to get a `TF-IDF` matrix. (Note, now `qs` and `sents` are converted to TF-IDF vectors in the same dimension. As a result, you can measure their similarities.) 
- Split the `TF-IDF` matrix into two sub matrices, one corresponding to `qs` and the other for `sents`. 
- Next, calculate the pairwise cosine similarity between the `qs` and `sents`. With $m$ questions and $n$ sentences, you should get a $m \times n$ matrix. (hint: you can `sklearn.metrics.pairwise_distances` to calculate pairwise distances between two matrices)
- Finally, the answer to each question is the sentence which has the `maximum similarity` to it. 
- Print out each question and its matched answer. Check if your QA system is able to find the right answer.(Depending on the packages you use, your answer might be different from mine.)

In [15]:
# Define the function
import sklearn
from sklearn.metrics import pairwise_distances

def find_solutions(qs, article):
    
    
    return mylist
    # add your code
                                      

In [16]:
# Test the system

find_solutions(qs, article)

NameError: name 'mylist' is not defined